# Fully learned reconstruction

In the following exercise we will train a notebook for solving the directly the Inverse Problem



## CT Operator matrix

In [ ]:
import numpy as np
from skimage.transform import radon, iradon

n, m = 28, 28

###############################################
# TODO: Set number of angles
angles = ...
###############################################

detectors = 40

theta = np.linspace(0.0, 180.0, angles, endpoint=False)

In [ ]:
import torch
import matplotlib.pyplot as plt

from torchvision import datasets, transforms
from torch.utils.data import Dataset

%matplotlib inline

batch_size = 64

transform_train = transforms.Compose([
    transforms.ToTensor()
])

transform_test = transforms.Compose([
    transforms.ToTensor()
])


def forward(phantom):
    sinogram = radon(phantom.squeeze(), theta, circle=False)
    sinogram += 0.1 * np.random.normal(size=sinogram.shape)
    return sinogram


class SinogramData(Dataset):
    def __init__(self, dataset, forward):
        super(SinogramData, self).__init__()
        self.dataset = dataset
        self.forward = forward
    
    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        x, _ = self.dataset[idx]
        x = np.array(x)
        y = self.forward(x)
        return x, y
    
# datasets (MNIST)
mnist_train = SinogramData(datasets.MNIST('/data', train=True, download=True, transform=transform_train), forward)
mnist_test  = SinogramData(datasets.MNIST('/data', train=False, download=True, transform=transform_test), forward)

# dataloaders
train_loader = torch.utils.data.DataLoader(mnist_train, batch_size=batch_size)
test_loader = torch.utils.data.DataLoader(mnist_test, batch_size=batch_size)


def visualize_batch(batch, ncols=8):
    nrows = (batch.shape[0] + ncols - 1) // ncols
    plt.figure(figsize=(15, 2 * nrows))
    for i in range(batch.shape[0]):
        plt.subplot(nrows, ncols, i+1)
        plt.imshow(batch[i].squeeze())
        plt.axis('off')
    plt.show()


x_batch, y_batch = next(iter(train_loader))
visualize_batch(x_batch)
visualize_batch(y_batch)

## Create the neural network

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

###############################################
# TODO: Add more layers (after running the first time)
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.lin = nn.Linear(angles * detectors, 28*28)

    def forward(self, x):
        x = self.lin(x.view(-1, angles * detectors))
        return x.view(-1, 1, 28, 28)
###############################################

    
model = Net()

## Set the hyperparameters, the loss function and the optimizer

In [ ]:
# learning rate
lr = 0.001

# number of epochs
n_epochs = 10

##################  Change these lines  ####################################
# TODO: Choose the loss function

lossf = torch.nn.MSELoss()

##################  Finish changing here  ##################################


optimizer = torch.optim.Adam(model.parameters(), lr=lr)

# set device to use
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
model.to(device)

## Train the network

In [ ]:
from skimage.measure import compare_psnr

dtype = torch.cuda.FloatTensor
# iterate on epochs
for i in range(n_epochs):
  
    train_loss = 0.0
    test_loss = 0.0


    model.train()
    # iterate on batches
    for j, (x, y) in enumerate(train_loader):
        x, y = x.type(dtype), y.type(dtype)
        
        ######################################################
        # TODO: Compute the output and do the backprop
        ...
        loss = ...
        ...
        ######################################################

        train_loss += loss.item() * x.shape[0]
            
        print('%d/%d' % (j, len(train_loader)), end='\r')
      
    
    model.eval()
    # iterate on batches
    for x, y in test_loader:
        x, y = x.type(dtype), y.type(dtype)
        ######################################################
        # TODO: Evaluate the performance of the current parameters
        ...
        loss = ...
        ...
        ######################################################
        outputs = model(y)
        loss = lossf(outputs, x)
        test_loss += loss.item() * x.shape[0]

    train_loss /= len(mnist_train)
    test_loss /= len(mnist_test)

    print('epoch: %d train-error: %.5f test-error: %.4f' % (i, train_loss, test_loss))

## Evaluate the performance (PSNR)

In [ ]:
def PSNR(imtrue, imother):
    return compare_psnr(imtrue, imother, data_range=imtrue.max()-imtrue.min())

model.eval()
# iterate on batches

psnr = 0
for x, y in test_loader:
    x, y = x.type(dtype), y.type(dtype)
    outputs = model(y)
    
    for j in range(x.shape[0]):
        psnr += PSNR(x[j].cpu().numpy(), outputs[j].detach().cpu().numpy())
        
psnr /= len(mnist_test)
print(psnr)

## Show some examples

In [ ]:
import torch.nn.functional as F
from skimage.transform import iradon


model.to(device)
model.eval()

########################################################
# TODO get first batch of data pairs
x, y = ...
########################################################

########################################################
# TODO compute the output of the network
outputs = ...
########################################################

for i in range(x.shape[0]):
    plt.figure(figsize=(15, 4))
   
    plt.subplot(1,3,1)
    
    plt.imshow(x[i].squeeze())
    plt.axis('off')
    
    plt.subplot(1,3,2)
    
    x_rec = iradon(y[i].cpu().numpy(), theta, circle=False)
    plt.imshow(x_rec)
    psnr1 = PSNR(x[i][0].cpu().numpy(), x_rec)
    
    plt.title('PSNR: %.2f' % psnr1)
    plt.axis('off')

    plt.subplot(1,3,3)
    psnr2 = PSNR(x[i].cpu().numpy(), outputs[i].detach().cpu().numpy())
    plt.title('PSNR: %.2f' % psnr2)
    
    plt.imshow(outputs[i].squeeze())
    plt.axis('off')
    
    plt.show()